## Low-Rank Autoregressive Tensor Completion (LATC)

This notebook shows how to implement a LATC (with truncated nuclear norm) imputer on some real-world traffic data sets. To overcome the problem of missing values within multivariate time series data, this method takes into account both low-rank structure and time series regression. For an in-depth discussion of LATC, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Mengying Lei, Nicolas Saunier, Lijun Sun (2021). <b>A Low-Rank Autorgressive Tensor Completion for Spatiotemporal Traffic Data Imputation</b>. arXiv:xxxx.xxxxx. <a href="https://arxiv.org/abs/xxxx.xxxxx" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


### Define LATC-imputer kernel

We start by introducing some necessary functions that relies on `Numpy`.

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>ten2mat</code>:</b> <font color="black">Unfold tensor as matrix by specifying mode.</font></li>
<li><b><code>mat2ten</code>:</b> <font color="black">Fold matrix as tensor by specifying dimension (i.e, tensor size) and mode.</font></li>
<li><b><code>svt_tnn</code>:</b> <font color="black">Implement the process of Singular Value Thresholding (SVT).</font></li>
</ul>
</div>

In [58]:
import numpy as np

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [59]:
def svt_tnn(mat, tau, theta):
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[: theta] = 1
        mid[theta : idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:, : idx] @ np.diag(mid)) @ (u[:, : idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[: idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, : idx] @ np.diag(vec) @ v[: idx, :]

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [60]:
def compute_mae(var, var_hat):
    return np.mean(np.abs(var - var_hat))

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

How to create $\boldsymbol{\Psi}_{0},\boldsymbol{\Psi}_{1},\ldots,\boldsymbol{\Psi}_{d}$?

In [61]:
from scipy import sparse
from scipy.sparse.linalg import spsolve as spsolve

def generate_Psi(dim_time, time_lags):
    Psis = []
    max_lag = np.max(time_lags)
    for i in range(len(time_lags) + 1):
        row = np.arange(0, dim_time - max_lag)
        if i == 0:
            col = np.arange(0, dim_time - max_lag) + max_lag
        else:
            col = np.arange(0, dim_time - max_lag) + max_lag - time_lags[i - 1]
        data = np.ones(dim_time - max_lag)
        Psi = sparse.coo_matrix((data, (row, col)), shape = (dim_time - max_lag, dim_time))
        Psis.append(Psi)
    return Psis

In [62]:
import numpy as np

# Example
dim_time = 5
time_lags = np.array([1, 3])
Psis = generate_Psi(dim_time, time_lags)
print('Psi_0:')
print(Psis[0].toarray())
print()
print('Psi_1:')
print(Psis[1].toarray())
print()
print('Psi_2:')
print(Psis[2].toarray())
print()

Psi_0:
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Psi_1:
[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]

Psi_2:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]



The main idea behind LATC-imputer is to approximate partially observed data with both low-rank structure and time series dynamics. The following `imputer` kernel includes some necessary inputs:

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>dense_tensor</code>:</b> <font color="black">This is an input which has the ground truth for validation. If this input is not available, you could use <code>dense_tensor = sparse_tensor.copy()</code> instead.</font></li>
<li><b><code>sparse_tensor</code>:</b> <font color="black">This is a partially observed tensor which has many missing entries.</font></li>
<li><b><code>time_lags</code>:</b> <font color="black">Time lags, e.g., <code>time_lags = np.array([1, 2, 3])</code>. </font></li>
<li><b><code>alpha</code>:</b> <font color="black">Weights for tensors' nuclear norm, e.g., <code>alpha = np.ones(3) / 3</code>. </font></li>
<li><b><code>rho</code>:</b> <font color="black">Learning rate for ADMM, e.g., <code>rho = 0.0005</code>. </font></li>
<li><b><code>lambda0</code>:</b> <font color="black">Weight for time series regressor, e.g., <code>lambda0 = 5 * rho</code></font></li>
<li><b><code>epsilon</code>:</b> <font color="black">Stop criteria, e.g., <code>epsilon = 0.0001</code>. </font></li>
<li><b><code>maxiter</code>:</b> <font color="black">Maximum iteration to stop algorithm, e.g., <code>maxiter = 100</code>. </font></li>
</ul>
</div>

In [63]:
def imputer(dense_mat, sparse_mat, time_lags, rho0, lambda0, theta, epsilon, maxiter, K = 3, pos_missing = None):
    """Low-Rank Autoregressive Matrix Completion, LAMC-imputer."""
    
    dim = np.array(sparse_mat.shape)
    d = len(time_lags)
    max_lag = np.max(time_lags)
    #pos_missing = np.where(sparse_mat == 0)
    #pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    pos_test = pos_missing
    dense_test = dense_mat[pos_test]
    del dense_mat
    
    T = np.zeros(dim)
    Z = sparse_mat.copy()
    Z[pos_missing] = np.mean(sparse_mat[sparse_mat != 0])
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim[1], time_lags)
    iden = sparse.coo_matrix((np.ones(dim[1]), (np.arange(0, dim[1]), np.arange(0, dim[1]))), shape = (dim[1], dim[1]))
    it = 0
    ind = np.zeros((d, dim[1] - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim[1] - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    while True:
        B = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B.append(Psis0[0] - sum(Psis0[1 :]))
        for k in range(K):
            rho = min(rho * 1.05, 1e5)
            X = svt_tnn(Z - T / rho, 1 / rho, theta)
            temp0 = rho / lambda0 * (X + T / rho)
            mat = np.zeros(dim)
            for m in range(dim[0]):
                mat[m, :] = spsolve(B[m].T @ B[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            T = T + rho * (X - Z)
        for m in range(dim[0]):
            Vm = Z[m, ind].T
            A[m, :] = np.linalg.pinv(Vm) @ Z[m, max_lag :]
        tol = np.linalg.norm((X - last_mat), 'fro') / snorm
        last_mat = X.copy()
        it += 1
        if it % 200 == 0:
            print('Iter: {}'.format(it))
            print('Tolerance: {:.6}'.format(tol))
            print('MAE: {:.6}'.format(compute_mae(dense_test, X[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, X[pos_test])))
            print()
        if (tol < epsilon and tol != 0) or (it >= maxiter):
            break

    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    print('Imputation MAE: {:.6}'.format(compute_mae(dense_test, X[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, X[pos_test])))
    print()
    
    return X

> We use `spslove` of `scipy.sparse.linalg` for updating $\boldsymbol{Z}$ because computing the inverse of a large matrix directly is computationally expensive.

## Arrival

In [64]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

def read_raw_data(name):
    data = pd.read_csv(name, index_col=0, parse_dates=True)
    columns_name = list(data.columns)
    index = data.index
    ##minmax-score
    norlizer = MinMaxScaler().fit(data)
    data = norlizer.transform(data)
    return data, columns_name, index, norlizer

def evaluate(raw_data, prediction, mask=None, seasonal=None):
    #rmse = RMSE_Metric(raw_data, miss_data, np.concatenate((miss_data[0,:].reshape(1,-1), prediction), axis=0), missing_flag=missing_flag)
    #print('RMSE:', rmse)
    prediction = prediction[mask].flatten()
    missed_raw_data = raw_data[mask].flatten()
    if seasonal is not None:
        missed_seasonal = seasonal[mask].flatten()
        concated_prediction += missed_seasonal
        missed_raw_data += missed_seasonal

    rmse = np.sqrt(mean_squared_error(missed_raw_data, prediction))
    mae = mean_absolute_error(missed_raw_data, prediction)
    #mape = mean_absolute_percentage_error(missed_raw_data, concated_prediction)
    #print('RMSE2:', rmse2)
    return (rmse, mae), prediction#, mape

In [68]:
import time
import numpy as np
import scipy.io
from pathlib import Path
np.random.seed(1000)

test_len = 12
#log_dir = Path('../logs/baselines/')
#log_dir.mkdir(exist_ok=True)
root_dir = Path('../..')

datasets = {
    'AU': '../data/open-data/HK2012-2018/Australia.csv',
    'PH': '../data/open-data/HK2012-2018/Philippine.csv',
    'SG': '../data/open-data/HK2012-2018/Singapore.csv',
    'TH': '../data/open-data/HK2012-2018/Thailand.csv',
    'UK': '../data/open-data/HK2012-2018/United_Kingdom.csv',
    'US': '../data/open-data/HK2012-2018/United_States.csv',
}
masks = {
    'random5': '../data/masks/random5.npy',
    'random10': '../data/masks/random10.npy',
    'block5': '../data/masks/block5.npy',
    'block10': '../data/masks/block10.npy'
}

dataset = 'AU'
mask_name = 'random5'

data, columns_name, index, norlizer = read_raw_data(root_dir/datasets[dataset])
mask = np.load(root_dir/masks[mask_name]).astype(bool)
miss_data = data.copy()
miss_data[mask] = 0
train_miss_data = miss_data[:-test_len].T
eval_real_data = data[:-test_len].T

In [69]:
for c in [1/10, 1/5, 1, 5, 10]:
    for theta in [5]:
        start = time.time()
        #time_lags = np.arange(1, 12)
        time_lags = np.array([1,12])
        rho = 1e-4
        lambda0 = c * rho
        print(c)
        print(theta)
        epsilon = 1e-4
        maxiter = 100
        mat_hat = imputer(eval_real_data, train_miss_data, time_lags, rho, lambda0, theta, epsilon, maxiter, pos_missing=mask[:-test_len].T)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

0.1
5
Total iteration: 88
Tolerance: 9.38856e-05
Imputation MAE: 0.132277
Imputation RMSE: 0.204676

Running time: 32 seconds

0.2
5
Total iteration: 88
Tolerance: 8.68472e-05
Imputation MAE: 0.132319
Imputation RMSE: 0.204618

Running time: 32 seconds

1
5
Total iteration: 88
Tolerance: 9.25411e-05
Imputation MAE: 0.132242
Imputation RMSE: 0.204608

Running time: 32 seconds

5
5


KeyboardInterrupt: 